In [ ]:
# Default parameters (will be overridden by Papermill)
model_type = 'linear_regression'
output_dir = 'outputs'
generate_plots = True

In [ ]:
import pandas as pd
import numpy as np
import os
import json
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

print(f"Evaluating model: {model_type}")
print(f"Generate plots: {generate_plots}")

In [ ]:
# Load model and metrics
model_dir = os.path.join(output_dir, 'models')
predictions_dir = os.path.join(output_dir, 'predictions')

with open(os.path.join(model_dir, f'{model_type}_model.pkl'), 'rb') as f:
    model = pickle.load(f)

with open(os.path.join(model_dir, f'{model_type}_metrics.json'), 'r') as f:
    metrics = json.load(f)

# Load predictions
train_pred = pd.read_csv(os.path.join(predictions_dir, f'{model_type}_train_predictions.csv'))
test_pred = pd.read_csv(os.path.join(predictions_dir, f'{model_type}_test_predictions.csv'))

print(f"Model loaded successfully")
print(f"Metrics loaded successfully")

In [ ]:
# Display detailed metrics
print(f"\n{'='*50}")
print(f"MODEL EVALUATION REPORT: {model_type.upper()}")
print(f"{'='*50}")

print(f"\nModel Configuration:")
print(f"  - Type: {metrics['model_type']}")
print(f"  - Alpha: {metrics['alpha']}")
print(f"  - Fit Intercept: {metrics['fit_intercept']}")
print(f"  - Training Time: {metrics['training_time_seconds']:.4f}s")

print(f"\nTraining Set Metrics:")
for key, value in metrics['train_metrics'].items():
    print(f"  - {key.upper()}: {value:.6f}")

print(f"\nTest Set Metrics:")
for key, value in metrics['test_metrics'].items():
    print(f"  - {key.upper()}: {value:.6f}")

if metrics['coefficients']:
    print(f"\nModel Coefficients (first 5):")
    for i, coef in enumerate(metrics['coefficients'][:5]):
        print(f"  - Feature {i}: {coef:.6f}")
    if len(metrics['coefficients']) > 5:
        print(f"  ... and {len(metrics['coefficients']) - 5} more")

if metrics['intercept']:
    print(f"\nIntercept: {metrics['intercept']:.6f}")

In [ ]:
# Calculate residuals
train_residuals = train_pred['actual'] - train_pred['predicted']
test_residuals = test_pred['actual'] - test_pred['predicted']

print(f"\nResidual Analysis:")
print(f"\nTraining Set Residuals:")
print(f"  - Mean: {train_residuals.mean():.6f}")
print(f"  - Std Dev: {train_residuals.std():.6f}")
print(f"  - Min: {train_residuals.min():.6f}")
print(f"  - Max: {train_residuals.max():.6f}")

print(f"\nTest Set Residuals:")
print(f"  - Mean: {test_residuals.mean():.6f}")
print(f"  - Std Dev: {test_residuals.std():.6f}")
print(f"  - Min: {test_residuals.min():.6f}")
print(f"  - Max: {test_residuals.max():.6f}")

In [ ]:
# Generate plots if enabled
if generate_plots:
    print(f"\nGenerating visualization plots...")
    plots_dir = os.path.join(output_dir, 'plots')
    os.makedirs(plots_dir, exist_ok=True)
    
    # Set style
    sns.set_style("whitegrid")
    
    # Plot 1: Actual vs Predicted (Train)
    plt.figure(figsize=(10, 6))
    plt.scatter(train_pred['actual'], train_pred['predicted'], alpha=0.5, label='Predictions')
    plt.plot([train_pred['actual'].min(), train_pred['actual'].max()],
             [train_pred['actual'].min(), train_pred['actual'].max()],
             'r--', lw=2, label='Perfect Prediction')
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(f'{model_type} - Training Set: Actual vs Predicted')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(plots_dir, f'{model_type}_train_actual_vs_predicted.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    # Plot 2: Actual vs Predicted (Test)
    plt.figure(figsize=(10, 6))
    plt.scatter(test_pred['actual'], test_pred['predicted'], alpha=0.5, color='orange', label='Predictions')
    plt.plot([test_pred['actual'].min(), test_pred['actual'].max()],
             [test_pred['actual'].min(), test_pred['actual'].max()],
             'r--', lw=2, label='Perfect Prediction')
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(f'{model_type} - Test Set: Actual vs Predicted')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(plots_dir, f'{model_type}_test_actual_vs_predicted.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    # Plot 3: Residuals Distribution
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    axes[0].hist(train_residuals, bins=30, alpha=0.7, color='blue', edgecolor='black')
    axes[0].set_xlabel('Residuals')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title(f'{model_type} - Training Set Residuals Distribution')
    axes[0].grid(True, alpha=0.3)
    
    axes[1].hist(test_residuals, bins=30, alpha=0.7, color='orange', edgecolor='black')
    axes[1].set_xlabel('Residuals')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title(f'{model_type} - Test Set Residuals Distribution')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(plots_dir, f'{model_type}_residuals_distribution.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    # Plot 4: Residuals vs Predicted
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    axes[0].scatter(train_pred['predicted'], train_residuals, alpha=0.5, color='blue')
    axes[0].axhline(y=0, color='r', linestyle='--', lw=2)
    axes[0].set_xlabel('Predicted Values')
    axes[0].set_ylabel('Residuals')
    axes[0].set_title(f'{model_type} - Training Set: Residuals vs Predicted')
    axes[0].grid(True, alpha=0.3)
    
    axes[1].scatter(test_pred['predicted'], test_residuals, alpha=0.5, color='orange')
    axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
    axes[1].set_xlabel('Predicted Values')
    axes[1].set_ylabel('Residuals')
    axes[1].set_title(f'{model_type} - Test Set: Residuals vs Predicted')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(plots_dir, f'{model_type}_residuals_vs_predicted.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"\nPlots saved to {plots_dir}")
    print(f"Files created:")
    print(f"  - {model_type}_train_actual_vs_predicted.png")
    print(f"  - {model_type}_test_actual_vs_predicted.png")
    print(f"  - {model_type}_residuals_distribution.png")
    print(f"  - {model_type}_residuals_vs_predicted.png")
else:
    print(f"\nPlot generation disabled")

In [ ]:
# Create evaluation summary
evaluation_summary = {
    'model_type': model_type,
    'evaluation_timestamp': pd.Timestamp.now().isoformat(),
    'metrics': metrics,
    'residual_analysis': {
        'train': {
            'mean': float(train_residuals.mean()),
            'std': float(train_residuals.std()),
            'min': float(train_residuals.min()),
            'max': float(train_residuals.max())
        },
        'test': {
            'mean': float(test_residuals.mean()),
            'std': float(test_residuals.std()),
            'min': float(test_residuals.min()),
            'max': float(test_residuals.max())
        }
    }
}

eval_path = os.path.join(output_dir, 'models', f'{model_type}_evaluation_summary.json')
with open(eval_path, 'w') as f:
    json.dump(evaluation_summary, f, indent=2)

print(f"\nEvaluation summary saved to {eval_path}")
print(f"\nEvaluation completed successfully!")